In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression


In [14]:

def longstaff_schwartz_algorithm(paths, payoffs):
    num_paths, num_steps = paths.shape
    
    # Initialize the cash flow matrix
    cash_flows = np.zeros_like(paths)
    cash_flows[:, -1] = payoffs[:, -1]  # Set the last column as the final payoffs
    
    # Perform backward induction
    for t in range(num_steps - 2, -1, -1):
        # Select in-the-money paths at time t
        in_the_money_paths = paths[payoffs[:, t] > 0, t+1:]
        in_the_money_payoffs = payoffs[payoffs[:, t] > 0, t+1:]
        if len(in_the_money_paths) > 0:
            # Calculate the continuation value using regression
            X = in_the_money_paths
            y = cash_flows[payoffs[:, t] > 0, t+1:]
            
            regression_model = LinearRegression()
            regression_model.fit(X, y)
            continuation_values = regression_model.predict(X)
            
            # Compare the continuation value with the immediate exercise value
            exercise_values = payoffs[payoffs[:, t] > 0, t]
            print(exercise_values)
            exercise_paths = in_the_money_paths[np.argmax(exercise_values, axis=0), :]
            
            # Update the cash flows matrix
            cash_flows[payoffs[:, t] > 0, t] = np.maximum(exercise_values, continuation_values.flatten())
            cash_flows[payoffs[:, t] > 0, t+1:] = 0
            
            # Set the future cash flows of exercised paths to zero
            cash_flows[np.arange(num_paths)[payoffs[:, t] > 0], t+1:] = 0
    
    # Discount the cash flows to the present value
    discount_factors = np.exp(-r * dt * np.arange(num_steps))
    present_value = np.mean(np.sum(cash_flows * discount_factors, axis=1))
    
    return present_value

In [15]:

# Set the parameters
r = 0.05  # Risk-free interest rate
T = 1.0  # Time to expiration
S0 = 100  # Initial asset price
K = 100  # Strike price
sigma = 0.2  # Volatility
num_paths = 1000  # Number of simulated paths
num_steps = 100  # Number of time steps

# Generate simulated price paths using geometric Brownian motion
dt = T / num_steps
t = np.linspace(0, T, num_steps)
drift = (r - 0.5 * sigma**2) * dt
diffusion = sigma * np.sqrt(dt) * np.random.randn(num_paths, num_steps)
paths = S0 * np.exp(np.cumsum(drift + diffusion, axis=1))

# Calculate the payoffs for the American call option
payoffs = np.maximum(paths - K, 0)

# Price the American call option using the Longstaff-Schwartz algorithm
option_price = longstaff_schwartz_algorithm(paths, payoffs)

# Print the option price
#print("The price of the American call option is:", option_price)


[1.54241985e+01 1.44660964e+01 6.43016460e+00 2.11670104e+01
 3.60892089e+01 2.83119532e+01 2.15706110e+01 1.06359785e+01
 1.70995040e+01 2.04323999e-01 2.69241454e+01 6.34853724e+00
 1.50361190e+01 9.25187362e+00 1.57942333e+01 3.05721942e+00
 3.63386116e+01 6.35393327e+00 4.01608240e+00 3.03003523e+01
 1.53045006e+01 3.65389251e-01 6.54342432e+00 1.36175678e+01
 9.18354874e+00 5.06030469e+01 2.07477195e+01 8.54732625e+00
 1.41840028e+00 2.25408535e+01 5.30773260e+00 9.22531062e+00
 3.83899398e+01 3.91530067e+01 5.88335752e-02 4.57525768e+01
 3.02011174e+01 5.61432677e-01 3.34821750e+01 4.88823281e+00
 4.02110630e+01 1.16384583e+01 1.56427147e+01 8.53996221e+00
 1.82854832e+01 3.33828585e+01 7.14298117e+00 8.95703188e+00
 9.12853484e+00 4.10683547e+01 8.56053259e+00 3.00378950e+01
 1.47351792e+01 1.89347075e+01 2.75500945e+01 1.63700343e+01
 2.37778183e+01 2.65967190e+01 1.91453862e+01 1.62458072e+01
 2.60393602e+01 2.40843235e+00 3.03038632e+00 2.94595319e+01
 3.28440217e+00 5.280489

[16.55302493 12.9123917   8.96024499 21.28189258 34.3384012  28.39538478
 21.73569282  8.37729654 18.38719883 24.80721658  6.51662062 17.64855373
 10.43585909 15.54394095  3.49665994 37.50198455  7.40257943  5.28819546
 28.63714275  9.91965305  1.48278075  1.37243315  4.57999281 12.29769809
 10.97440128 49.04210469 17.38310718  8.59542462 25.22547638  3.07810765
  8.84321021 38.79312761 38.88952401  0.35458337 45.25833393 31.68493678
  2.36425387 33.92454098  4.89664121 46.52642599 11.51110257 15.38686492
 12.18921368  0.66964593 17.15723867 36.49182267  7.36597053 10.49448446
  6.12921802 39.89425197  6.23414675 34.52159211  5.66496457 15.08675656
 20.03725759 32.7871169  18.63897736 27.46112151 26.65092929 18.84132107
 16.91380065 26.97182269  1.16609833 29.48785076  5.40328021 32.91395741
  9.75112555  0.60272439 17.37035308 27.05429729 38.18079431 27.05149858
  9.77126801  5.8818656  12.90537268 10.21989957 24.1587071   8.46549922
  4.27777829 46.38536435 17.15137997 33.00792735 19

ValueError: operands could not be broadcast together with shapes (540,) (1080,) 

In [11]:
payoffs


array([[ 2.17232756,  3.91082906,  4.45484972, ...,  4.77635351,
         6.45299726,  6.44668244],
       [ 4.3695503 ,  6.25230713,  7.60596053, ..., 13.14038264,
        14.52060992, 14.44593005],
       [ 0.        ,  0.        ,  2.54284191, ...,  0.10371964,
         1.31702331,  1.36372594],
       ...,
       [ 2.46335444,  2.33479231,  1.18631975, ...,  0.        ,
         0.        ,  1.28660681],
       [ 2.7204202 ,  1.05237892,  3.4447093 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.29646656,  0.56804824, ..., 32.73414624,
        35.68798246, 43.72348351]])

## Binomiale Tree

In [17]:
import numpy as np

# Set the parameters
r = 0.05  # Risk-free interest rate
T = 1.0  # Time to expiration
S0 = 100  # Initial asset price
K = 100  # Strike price
sigma = 0.2  # Volatility
num_steps = 100  # Number of time steps

# Calculate the parameters for the binomial tree
dt = T / num_steps
u = np.exp(sigma * np.sqrt(dt))  # Up factor
d = 1 / u  # Down factor
p = (np.exp(r * dt) - d) / (u - d)  # Probability of up move

# Initialize the stock price and option value arrays
stock_prices = np.zeros((num_steps+1, num_steps+1))
option_values = np.zeros((num_steps+1, num_steps+1))

# Calculate the stock prices at each node of the binomial tree
for i in range(num_steps+1):
    for j in range(i+1):
        stock_prices[j, i] = S0 * (u**j) * (d**(i-j))

# Calculate the option values at the final time step
option_values[:, num_steps] = np.maximum(stock_prices[:, num_steps] - K, 0)

# Perform backward induction
for i in range(num_steps-1, -1, -1):
    for j in range(i+1):
        # Calculate the expected option value at each node
        expected_value = np.exp(-r * dt) * (p * option_values[j, i+1] + (1 - p) * option_values[j+1, i+1])
        
        # Calculate the immediate exercise value at each node
        immediate_value = np.maximum(stock_prices[j, i] - K, 0)
        
        # Determine whether to exercise the option or continue
        option_values[j, i] = np.maximum(expected_value, immediate_value)

# The option price is the value at the root node of the binomial tree
option_price = option_values[0, 0]

# Print the option price
print("The price of the American option using the binomial tree is:", option_price)


The price of the American option using the binomial tree is: 7.243554844907945


Difference finie

In [25]:


# Set the parameters
r = 0.05  # Risk-free interest rate
T = 1.0  # Time to expiration
S0 = 100  # Initial asset price
K = 100  # Strike price
sigma = 0.2  # Volatility
num_steps = 100  # Number of time steps
num_nodes = num_steps + 1  # Number of nodes in the grid

# Calculate the grid size and time step
dt = T / num_steps
dS = S0 / np.sqrt(num_steps)

# Initialize the grid for stock prices and option values
stock_prices = np.zeros((num_nodes, num_nodes))
option_values = np.zeros((num_nodes, num_nodes))

# Set the boundary conditions at expiration time
stock_prices[:, num_steps] = np.arange(num_nodes) * dS
option_values[:, num_steps] = np.maximum(stock_prices[:, num_steps] - K, 0)

# Perform backward induction
for i in range(num_steps-1, -1, -1):
    for j in range(1, num_steps):
        # Calculate the coefficients for the finite difference equation
        a = 0.5 * dt * (sigma**2 * j**2 - r * j)
        b = 1 - dt * (sigma**2 * j**2 + r)
        c = 0.5 * dt * (sigma**2 * j**2 + r * j)
        
        # Calculate the option value at each node
        option_values[j, i] = a * option_values[j-1, i+1] + b * option_values[j, i+1] + c * option_values[j+1, i+1]
        
        # Check for early exercise
        immediate_value = stock_prices[j, i] - K
        option_values[j, i] = max(option_values[j, i], immediate_value)

# The option price is the value at the center of the grid
option_price = option_values[num_steps // 2, 0]

# Print the option price
print("The price of the American option using finite difference method is:", option_price)


The price of the American option using finite difference method is: 1.2930655026133882e+34


In [26]:
option_values

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.32047030e-09,  1.20242929e-09,  1.09373476e-09, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.87232451e-07,  2.64718225e-07,  2.43730871e-07, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.33870505e+55, -1.00000000e+02,  1.10851704e+54, ...,
         8.80099975e+02,  8.80050000e+02,  8.80000000e+02],
       [-1.00000000e+02,  2.14547930e+54, -1.00000000e+02, ...,
        -1.00000000e+02,  8.90050000e+02,  8.90000000e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  9.00000000e+02]])